# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import getFamaFrenchFactors as gff
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import portfolio_returns as pr

# Importing Dataset
Here, Just importing the monthly data from the getfamaFrenchFactors as this project focuses on checking the investment returns, through monthly investment with strategic seasonal investment.

In [2]:
monthly_df = gff.famaFrench3Factor(frequency='m')
monthly_df.head()

,date_ff_factors,Mkt-RF,SMB,HML,RF
0,1926-07-31,0.0296,-0.0256,-0.0243,0.0022
1,1926-08-31,0.0264,-0.0117,0.0382,0.0025
2,1926-09-30,0.0036,-0.0140,0.0013,0.0023
3,1926-10-31,-0.0324,-0.0009,0.0070,0.0032
4,1926-11-30,0.0253,-0.0010,-0.0051,0.0031
